In [ ]:
#!pip install geemap
#!pip install ffmpeg-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.2 MB/s eta 0:00:00


In [ ]:
import os
import ee
import geemap
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

import urllib


from PIL import Image, ImageSequence

import calendar
from tqdm import tqdm

from ipywidgets import Button, Dropdown, VBox, Layout

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#!earthengine authenticate

In [ ]:
img_L= None
img_S = None

timelapse_L = None

#roi= None #no funciona

In [ ]:
ee.Initialize()# Inicializar Earth Engine

#url_mapa_base = "https://wms.ign.gob.ar/geoserver/gwc/service/tms/1.0.0/capabaseargenmap@EPSG%3A3857@png/%7Bz%7D/%7Bx%7D/%7B-y%7D.png"

Map = geemap.Map(center=(-40, -64), zoom=4)
#Map.add_basemap(url_mapa_base, attribution="Atribución de tu mapa base") #argenmap
Map.add_basemap('HYBRID') #Ejemplos de mapas base: ROADMAP, SATELLITE, TERRAIN, HYBRID
#Map

In [ ]:
def submit_limpiar_clicked (b): #limpieza
  # Limpieza de los valores de los widgets
    title.value = ""
    bands.value = 'Red/Green/Blue'
    start_year.value = 1984
    end_year.value = 2023
    start_month.value = 1
    end_month.value = 12
    speed.value = 3
    checkBox_desvanecimiento.value = False
    checkBox_nubes.value = False
    font_size.value = 30
    font_color.value = 'white'
    progress_bar_color.value = 'blue'
    nombre_gif.value = ""
    checkBox_descarga_gif.value = False
    checkBox_descarga_video.value = False

    output.clear_output()
    
    
  



#..............................................................................# Boton
# Evento de clic del botón
def submit_clicked(b):
  with output:
    #output.clear_output()
    clear_output()  # Limpiar el área de salida   
    print('PROCESANDO... POR FAVOR ESPERE A QUE EL GIF SEA GENERADO')

    #verificar periodo
    if start_year.value >= end_year.value:
      print('El año inicial debe ser menor que el año final.')
      return # salgo de la funcion
    if start_month.value > end_month.value:
      print('El mes inicial debe ser menor o igual que el mes final.')
      return # salgo de la funcion

    # Verificar si el Checkbox está marcado
    if checkBox_descarga_gif.value:
      if nombre_gif.value == "":
        clear_output()
        print('Debe darle un nombre a el archivo')
        return
      else:
        out_gif = f'{nombre_gif.value}.gif'

    else:
        out_gif = None  # Si el Checkbox no está marcado, no se establece el nombre del gif
    
    if checkBox_descarga_video.value:
      mp4=True
    else: 
      mp4=False

    if checkBox_desvanecimiento.value:
      fading=True #desvanecimiento
    else:
      fading=False #desvanecimiento

    if checkBox_nubes.value:
      apply_fmask=True #desvanecimiento
    else:
      apply_fmask=False #desvanecimiento


    

    try:
      timelapse_L = geemap.landsat_timelapse(
        roi=Map.user_roi,
        out_gif=out_gif,
        start_year=start_year.value,
        end_year=end_year.value,
        start_date=f'{start_month.value:02d}-01',
        end_date=f'{end_month.value:02d}-28',
        frequency='year',
        bands=bands.value.split('/'),
        frames_per_second=speed.value,
        title=title.value,
        font_color=font_color.value,
        font_size=font_size.value,
        progress_bar_color=progress_bar_color.value,
        mp4=mp4,
        apply_fmask=apply_fmask,
        fading=fading,
    )
    
      clear_output()  # Limpiar el área de salida
  
    # Verificar si el Checkbox está marcado
      if checkBox_descarga_gif.value:
      # Obtener la ruta actual del directorio de trabajo
        ruta_proyecto = os.getcwd()
        print(f'LA DESCARGA SE ENCUENTRA EN: {ruta_proyecto}\{out_gif}')
      else:
        pass

      if checkBox_descarga_video.value:
        # Obtener la ruta actual del directorio de trabajo
        ruta_proyecto = os.getcwd()
        print(f'LA DESCARGA SE ENCUENTRA EN: {ruta_proyecto}\{nombre_gif.value}.mp4')
      else:
        pass


# Muestra el timelapse en el mapa
      geemap.show_image(timelapse_L)


    except FileNotFoundError:
      print("Error: No se pudo encontrar el archivo de imagen GIF. Por favor, intenta nuevamente más tarde.")
    except Exception as e:
      print("Error inesperado:", e)  




#..............................................................................#


In [ ]:

#def download_timelapse(b):
#    # Descarga el archivo en el dispositivo local
#    geemap.download_ee_video(timelapse_L, out_gif='landsat_timelapse.gif')

#    #files.download('landsat.gif')
#    #os.rename('landsat.gif', '/ruta/local/deseada/landsat.gif')


In [ ]:
#titulo
label_titulo = widgets.HTML(
    value='<div style="display: flex; justify-content: center; align-items: center; height: 90%; margin: 0; margin-top: 0;">' + 
          '<b><p style="text-align:center; font-size:3em; margin: 0;">TIMELAPSE ANUAL (DIG - IGN)</p></b>' + 
          '</div>',
    layout=widgets.Layout(border='solid 1px black', width='98%', height='90px')  # Utilizamos porcentajes
)

display(label_titulo)

In [ ]:
#explicación
label_explicacion_1 = widgets.HTML(value="<p style='font-size: 20px; margin: 0;'>Esta herramienta permite generar un timelapse para una región, tomando una imagen por año a partir de un período en particular.</p>")
label_explicacion_2 = widgets.HTML(value="<p style='font-size: 20px; margin: 0;'>Se deberá dibujar en el mapa un polígono, completar los parámetros y luego crear el TIMELAPSE. El resultado puede ser descargado en formato .gif o .mp4.</p>")

caja_explicacion = widgets.VBox([label_explicacion_1, label_explicacion_2])

# Modificar el tamaño del VBox
caja_explicacion.layout.width = '90%'  # Ancho del 90% del contenedor padre
caja_explicacion.layout.height = '10%'  # Altura ajustada automáticamente al contenido

display(caja_explicacion)


In [ ]:
#..............................................................................# Interfaz grafica
style = {'description_width': 'initial'}

label_datos_landsat = widgets.HTML(value="<b style='font-size: 20px;'>Complete los parámetros para generar el timelpase con imágenes Landsat</b>")

title = widgets.Text(
    description='Título:', 
    value='', 
    #width=200, 
    style=style,  #Landsat Timelapse - DIG - IGN
    layout=widgets.Layout(width='40%')
)

bands = widgets.Dropdown(
    description='Combinación de bandas: ',
    options=[
        ('Color verdadero', 'Red/Green/Blue'),
        ('Color natural', 'NIR/Red/Green'),
        ('Falso color de vegetación', 'SWIR2/SWIR1/NIR'),
        ('Color de vegetación', 'NIR/SWIR1/Red'),
        ('Índice de vegetación de diferencia normalizada (NDVI)', 'SWIR2/NIR/Red'),
        ('Índice de agua (NDWI)', 'SWIR2/SWIR1/Red'),
        ('Índice de suelo (Soil Index)', 'SWIR1/NIR/Blue'),
        ('Índice de humedad del suelo (Soil Moisture Index)','NIR/SWIR1/Blue'),
        ('Índice de vegetación de ajuste de suelo (SAVI)', 'SWIR2/NIR/Green'),
        ('Índice de vegetación mejorado (EVI)','SWIR1/NIR/Red'),
    ],
    value='Red/Green/Blue',
    style=style,
    layout=widgets.Layout(width='55%')  # Ajusta el ancho del widget
)





hbox1 = widgets.HBox([title, bands])

#-------------------------------------------------------------------------------#
#label_periodo = widgets.HTML(value="<b style='font-size: 40px;'>Período de búsqueda de imágenes:</b>")
label_periodo = widgets.HTML(value="<b>Período de búsqueda de imágenes: </b>")

start_year = widgets.IntSlider(
    description='Año inicial:', value=1984, min=1984, max=2023, style=style,
)

end_year = widgets.IntSlider(
    description='Año final:', value=2023, min=1984, max=2023, style=style
)
#end_year.style.handle_color = 'lightblue'

hbox2 = widgets.HBox([start_year, end_year])


#-------------------------------------------------------------------------------#
start_month = widgets.IntSlider(
    description='Mes inicial:', value=1, min=1, max=12, style=style,
)

end_month = widgets.IntSlider(
    description='Mes final:', value=12, min=1, max=12, style=style
)
#end_year.style.handle_color = 'lightblue'

hbox2_5 = widgets.HBox([start_month, end_month])

#-------------------------------------------------------------------------------#
label_estilo = widgets.HTML(value="<b>Estilo: </b>")

speed = widgets.IntSlider(
  description='Frames por segundo:',
  tooltip='Frames por segundo:',
  value=3,
  min=1,
  max=10,
  style=style,
)

# Crear un widget Checkbox
checkBox_desvanecimiento = widgets.Checkbox(
    description='Desvanecimiento de imágenes',
    value=False,  # Inicialmente apagado
    style=style,
)

checkBox_nubes = widgets.Checkbox(
    description='Máscara de nubes',
    value=False,  # Inicialmente apagado
    style=style,
)


hbox3 = widgets.HBox([speed, checkBox_desvanecimiento, checkBox_nubes])

#-------------------------------------------------------------------------------#
font_size = widgets.IntSlider(
    description='Tamaño de fuente:', value=30, min=10, max=50, style=style
)

font_color = widgets.ColorPicker(
    concise=False, description='Color de fuente:', value='white', style=style
)

progress_bar_color = widgets.ColorPicker(
    concise=False, description='Color de la barra de progreso:', value='blue', style=style
)

hbox4 = widgets.HBox([font_size, font_color, progress_bar_color])


#-------------------------------------------------------------------------------#
label_descarga = widgets.HTML(value="<b>Descarga: </b>")

# Crear un widget de entrada de texto
nombre_gif = widgets.Text(
    description='Nombre del archivo:',
    value='', #Escribe un nombre para el archivo resultante, sin espacios ni carateres especiales.
    style=style,
    color='#FF0000',
    layout=widgets.Layout(width='50%')
)

# Crear un widget Checkbox
checkBox_descarga_gif = widgets.Checkbox(
    description='Descargar en formato gif',
    value=False,  # Inicialmente apagado
    style=style,
    layout=widgets.Layout(width='25%')
)


# Crear un widget Checkbox
checkBox_descarga_video = widgets.Checkbox(
    description='Descargar en formato video',
    value=False,  # Inicialmente apagado
    style=style,
    layout=widgets.Layout(width='25%')
)


hbox1_5 = widgets.HBox([nombre_gif, checkBox_descarga_gif, checkBox_descarga_video])



#-------------------------------------------------------------------------------#
# Crea la etiqueta
label = widgets.Label("") ## ?

#-------------------------------------------------------------------------------#

#----------  Botón  ----------#
# Establece el ancho del botón
#button_width = '170px'  # Puedes ajustar el valor según tus necesidades

submit = widgets.Button(
    description='Crear TIMELAPSE', ##ver
    button_style='primary',
    tooltip='Click en el botón para crear el timelapse con imágenes Landsat',
    icon='globe',
    layout=widgets.Layout(width='170px')
)

submit_limpiar = widgets.Button(
    description='Limpiar', ##ver
    button_style='primary',
    tooltip='Click en el botón para limpiar los parámetros',
    icon='eraser',
    layout=widgets.Layout(width='170px')
)

hbox_botones = widgets.HBox([submit, submit_limpiar])

#-------------------------------------------------------------------------------#
output = widgets.Output() #

#si se presiona el botón se llama a la función
submit.on_click(submit_clicked)

submit_limpiar.on_click(submit_limpiar_clicked)

#boton_descarga.on_click(download_timelapse)

#-------------------------------------------------------------------------------#
etiqueta_linea = widgets.Label(value='____________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________')


In [ ]:
#caja1= widgets.HBox([Map])
caja0 = widgets.VBox([etiqueta_linea, label_datos_landsat, label, hbox1, label, label_periodo, hbox2, hbox2_5, label, label_estilo, hbox3, hbox4,label, label_descarga, hbox1_5, label, hbox_botones, etiqueta_linea]) #imprimir interfaz

# Organizar el mapa y las herramientas en un diseño de cuadrícula
#grid_layout = widgets.GridBox([
#    Map,  # Mapa en la columna 0
#    caja1
#], layout=widgets.Layout(grid_template_columns='50% 50%'))


#display(grid_layout)

In [ ]:
texto = """
Combinación RGB para Landsat: <br>
- '<b>Red/Green/Blue</b>': Color verdadero<br>
- '<b>NIR/Red/Green</b>': Color natural<br>
- '<b>SWIR2/SWIR1/NIR</b>': Falso color de vegetación<br>
- '<b>NIR/SWIR1/Red</b>': Color de vegetación<br>
- '<b>SWIR2/NIR/Red</b>': Índice de vegetación de diferencia normalizada (NDVI)<br>
- '<b>SWIR2/SWIR1/Red</b>': Índice de agua (NDWI)<br>
- '<b>SWIR1/NIR/Blue</b>': Índice de suelo (Soil Index)<br>
- '<b>NIR/SWIR1/Blue</b>': Índice de humedad del suelo (Soil Moisture Index)<br>
- '<b>SWIR2/NIR/Green</b>': Índice de vegetación de ajuste de suelo (SAVI)<br>
- '<b>SWIR1/NIR/Red</b>': Índice de vegetación mejorado (EVI)<br>
"""

infoLandsat = widgets.HTML(
    value=texto,
)

caja1 = widgets.Tab([caja0, infoLandsat])
caja1.set_title(0, 'LANDSAT')
caja1.set_title(1, 'Información bandas')

caja1.layout.height = '640px' 

# Puedes ajustar el ancho según la longitud del texto
#ancho_texto = len('Información de bandas de LANDSAT') * 50  # Ajusta el valor multiplicador según sea necesario

# Establece el ancho mínimo para la solapa del índice 1
#caja1.set_title(1, 'Información de bandas de LANDSAT'.ljust(ancho_texto))

In [ ]:
# Organizar el mapa y las herramientas en un diseño de cuadrícula
grid_layout_v2 = widgets.GridBox([
    Map,  # Mapa en la columna 0
    caja1
], layout=widgets.Layout(grid_template_columns='50% 50%'))

# Modificar el alto del GridBox
#grid_layout_v2.layout.height = '150px'  # Puedes ajustar el alto según lo necesites

# Ajustar la alineación del contenido hacia arriba
grid_layout_v2.layout.justify_content = 'flex-start'

display(grid_layout_v2)

In [ ]:
output

Este código es una adaptación del que se encuentra disponible en el sitio web https://geemap.org/